# Testing WOFpy SOAP and REST for WOF/WaterML 1.1
Testing performed on cloud-based WOFpy endpoints with ODM2 timeseries MySQL and PostgreSQL backends. These resources are managed by WOFpy developers for WOFpy testing and validation.

8/19/2017. Don Setiawan and Emilio Mayorga

Updates:
- 8/21/2017 Added the printing of Exception, and spaces between requests

In [1]:
from urlparse import urljoin
import io

import requests
import ulmo
import sys, traceback

In [2]:
print(ulmo.cuahsi.wof.__doc__)


    ulmo.cuahsi.wof
    ~~~~~~~~~~~~~~~

    `CUAHSI WaterOneFlow`_ web services

    .. _CUAHSI WaterOneFlow: http://his.cuahsi.org/wofws.html



In [3]:
print([obj for obj in dir(ulmo.cuahsi.wof) if not obj.startswith('__')])

['absolute_import', 'core', 'get_site_info', 'get_sites', 'get_values', 'get_variable_info']


In [4]:
def get_exception(exc_info):
    exc_type, exc_value, exc_traceback = exc_info
    print('*** {}:'.format(exc_type.__name__))
    traceback.print_exception(exc_type, exc_value, exc_traceback, file=sys.stdout)
    print('')

## Endpoints and testing resources

In [5]:
def site_and_variable_codes(networkcode):
    if 'postgres' in networkcode:
        sitecode = '{netcode}:160065_4vars'.format(netcode=networkcode)
        variablecode = '{netcode}:EnviroDIY_Mayfly_Temp'.format(netcode=networkcode)
    elif 'mysql' in networkcode:
        sitecode = '{netcode}:USU-LBR-Mendon'.format(netcode=networkcode)
        variablecode = '{netcode}:USU36'.format(netcode=networkcode)
    
    return sitecode, variablecode

In [6]:
server_base_url = 'http://54.186.36.247:8080'
networkcodes = ['postgresqlodm2timeseries', 'mysqlodm2timeseries']

## SOAP 1.1 Testing

In [7]:
wofpy_wsdl_url = '/soap/cuahsi_1_1/.wsdl'
for networkcode in networkcodes:
    url = urljoin(server_base_url, networkcode+wofpy_wsdl_url)
    sitecode, variablecode = site_and_variable_codes(networkcode)

    print('--------------------------')
    print('Testing {}'.format(url))
    print('--------------------------')
        
    print('SITES-----------------')
    try:
        sites = ulmo.cuahsi.wof.get_sites(url)
        
        print(sites.keys())
        print('')
    except:
        print('GetSites failed!')
        get_exception(sys.exc_info())

    
    print('SITE INFO {}-------------'.format(sitecode))
    try:
        siteinfo = ulmo.cuahsi.wof.get_site_info(url, site_code=sitecode)
        
        print(siteinfo['site_property'])
        print('')
    except:
        print('GetSiteInfo failed!')
        get_exception(sys.exc_info())
    
    print('VARIABLES-------------')
    try:
        variables = ulmo.cuahsi.wof.get_variable_info(url)
        
        print(variables.keys())
        print('')
    except:
        print('GetVariables failed!')
        get_exception(sys.exc_info())

    print('VARIABLE INFO {}-------------'.format(variablecode))
    try:
        variable = ulmo.cuahsi.wof.get_variable_info(url, variable_code=variablecode)
        
        print(variable['name'])
        print('')
    except:
        print('GetVariableInfo failed!')
        get_exception(sys.exc_info())
    
        
    print('VALUES----------------')    
    try:
        values = ulmo.cuahsi.wof.get_values(url, site_code=sitecode, variable_code=variablecode)
        
        print(len(values['values']))
        print('')
    except:
        print('GetValues failed!')
        get_exception(sys.exc_info())

--------------------------
Testing http://54.186.36.247:8080/postgresqlodm2timeseries/soap/cuahsi_1_1/.wsdl
--------------------------
SITES-----------------
GetSites failed!
*** KeyError:
Traceback (most recent call last):
  File "<ipython-input-7-53857f5fcb02>", line 12, in <module>
    sites = ulmo.cuahsi.wof.get_sites(url)
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/cuahsi/wof/core.py", line 60, in get_sites
    sites = waterml.v1_1.parse_site_infos(response_buffer)
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/waterml/v1_1.py", line 9, in parse_site_infos
    content_io, WATERML_V1_1_NAMESPACE, site_info_names=['siteInfo', 'sourceInfo'])
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/waterml/common.py", line 108, in parse_site_infos
    for site_info_element in site_info_elements
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/waterml/common.py", line 324, in _parse_site_

## REST 1.1 Testing

In [8]:
wofpy_rest_url = '/rest/1_1/'
for networkcode in networkcodes:
    url = urljoin(server_base_url, networkcode+wofpy_rest_url)
    sitecode, variablecode = site_and_variable_codes(networkcode)

    print('--------------------------')
    print('Testing {}'.format(url))
    print('--------------------------')
    
    print('SITES-----------------')
    try:
        req = requests.get(urljoin(url, 'GetSites'))
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        sites = ulmo.waterml.v1_1.parse_site_infos(response_buffer)
        
        print(sites.keys())
        print('')
    except:
        print('GetSites failed!')
        get_exception(sys.exc_info())
    
    print('SITE INFO {}-------------'.format(sitecode))
    try:
        req = requests.get(urljoin(url, 'GetSiteInfo'), params={'site':sitecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        siteinfo = ulmo.waterml.v1_1.parse_sites(response_buffer)
        
        print(siteinfo[sitecode.split(':')[1]]['site_property'])
        print('')
    except:
        print('GetSiteInfo failed!')
        get_exception(sys.exc_info())
    
    print('VARIABLES-------------')
    try:
        req = requests.get(urljoin(url, 'GetVariables'))
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        variables = ulmo.waterml.v1_1.parse_variables(response_buffer)
        
        print(variables.keys())
        print('')
    except:
        print('GetVariables failed!')
        get_exception(sys.exc_info())

    print('VARIABLE INFO {}-------------'.format(variablecode))
    try:
        req = requests.get(urljoin(url, 'GetVariableInfo'), params={'variable':variablecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        variable = ulmo.waterml.v1_1.parse_variables(response_buffer)
        
        print(variable[variablecode.split(':')[1]]['name'])
        print('')
    except:
        print('GetVariableInfo failed!')
        get_exception(sys.exc_info())
    
        
    print('VALUES----------------')    
    try:
        req = requests.get(urljoin(url, 'GetValues'), params={'location':sitecode, 
                                                              'variable':variablecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        values = ulmo.waterml.v1_1.parse_site_values(response_buffer)
        
        print(len(values[variablecode.split(':')[1]]['values']))
        print('')
    except:
        print('GetValues failed!')
        get_exception(sys.exc_info())

--------------------------
Testing http://54.186.36.247:8080/postgresqlodm2timeseries/rest/1_1/
--------------------------
SITES-----------------
GetSites failed!
*** KeyError:
Traceback (most recent call last):
  File "<ipython-input-8-580ac2f81f0e>", line 14, in <module>
    sites = ulmo.waterml.v1_1.parse_site_infos(response_buffer)
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/waterml/v1_1.py", line 9, in parse_site_infos
    content_io, WATERML_V1_1_NAMESPACE, site_info_names=['siteInfo', 'sourceInfo'])
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/waterml/common.py", line 108, in parse_site_infos
    for site_info_element in site_info_elements
  File "//anaconda/envs/uwapl_em_mc_1aui/lib/python2.7/site-packages/ulmo/waterml/common.py", line 324, in _parse_site_info
    for note in site_info.findall(namespace + 'note')
  File "src/lxml/lxml.etree.pyx", line 2452, in lxml.etree._Attrib.__getitem__ (src/lxml/lxml.etree.c:68625)